In [1]:
import glob
import multiprocessing as mp
import os

from gesim import gesim
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [8]:
NUM_CPUS = 130


def calc_bulk_tanimoto_sim(query_fp, target_fps, output_dir, query_id):
    sim_results = DataStructs.BulkTanimotoSimilarity(query_fp, target_fps)
    out_list = [f"{i}\t{v}\n" for i, v in enumerate(sim_results)]
    ofname = os.path.join(output_dir, f"tanimoto_sims_active{query_id}.txt")
    with open(ofname, 'w') as f:
        f.writelines(out_list)


def calc_bulk_gesim(query_mol, target_mols, output_dir, query_id, r):
    sim_results = gesim.graph_entropy_similarity_batch(query_mol, target_mols, r=r)
    out_list = [f"{i}\t{v}\n" for i, v in enumerate(sim_results)]
    ofname = os.path.join(output_dir, f"gesims_r{r}_active{query_id}.txt")
    with open(ofname, 'w') as f:
        f.writelines(out_list)


def main():
    target_names = []
    for f in sorted(glob.glob("./data/*")):
        if os.path.isdir(f):
            target_names.append(os.path.basename(f))

    for target_name in target_names:
        #if target_name != "ESR1_ant":
        #    continue
        key_amol_dict = {}
        key_afp_dict = {}
        active_mols = []
        active_fps = []
        inactive_mols = []
        inactive_fps = []
        all_mols = None
        all_fps = None

        print(f"Target: {target_name} Process...")
        with open(f"./data/{target_name}/actives.smi", 'r') as f:
            lines = [l.split(' ')[0] for l in f.readlines()]
            active_mols = [Chem.MolFromSmiles(s) for s in lines[:10]]
            #active_mols = [Chem.MolFromSmiles(s) for s in lines]
            print(f"  #actives ({target_name}): {len(active_mols)}")
            print("    Morgan fingerprint calculation...")
            active_fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048) for m in active_mols]
            print("    Done")
            key_amol_dict = {k: m for k, m in enumerate(active_mols)}
            key_afp_dict = {k: fp for k, fp in enumerate(active_fps)}
        with open(f"./data/{target_name}/inactives.smi", 'r') as f:
            lines = [l.split(' ')[0] for l in f.readlines()]
            inactive_mols = [Chem.MolFromSmiles(s) for s in lines[:100]]
            #inactive_mols = [Chem.MolFromSmiles(s) for s in lines]
            print(f"  #inactives ({target_name}): {len(inactive_mols)}")
            print("    Morgan fingerprint calculation...")
            inactive_fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048) for m in inactive_mols]
            print("    Done")
        all_mols = active_mols + inactive_mols
        all_fps = active_fps + inactive_fps

        print("  Tanimoto similarity calculation...")
        output_tanimoto_dir = f"./result/tanimoto_sim/result_{target_name}/"
        os.makedirs(output_tanimoto_dir, exist_ok=True)
        num_cpus = len(active_fps) if len(active_fps) < NUM_CPUS else NUM_CPUS
        pool = mp.Pool(num_cpus)
        for query_id, query_afp in key_afp_dict.items():
            pool.apply_async(calc_bulk_tanimoto_sim, args=(query_afp, all_fps, output_tanimoto_dir, query_id))
        pool.close()
        pool.join()
        print("  Done")

        print("  Graph entropy similarity calculation...")
        output_gesim_dir = f"./result/ge_sim/result_{target_name}/"
        os.makedirs(output_gesim_dir, exist_ok=True)
        num_cpus = len(active_mols) if len(active_mols) < NUM_CPUS else NUM_CPUS
        radius_list = [1, 2, 3, 4]
        for r in radius_list:
            pool = mp.Pool(num_cpus)
            for query_id, query_amol in key_amol_dict.items():
                pool.apply_async(calc_bulk_gesim, args=(query_amol, all_mols, output_gesim_dir, query_id, r))
            pool.close()
            pool.join()
        print("  Done")

main()

Target: ADRB2 Process...
  #actives (ADRB2): 10
    Morgan fingerprint calculation...
    Done
  #inactives (ADRB2): 100
    Morgan fingerprint calculation...
    Done
  Tanimoto similarity calculation...
  Done
  Graph entropy similarity calculation...
  Done
Target: ALDH1 Process...
  #actives (ALDH1): 10
    Morgan fingerprint calculation...
    Done
  #inactives (ALDH1): 100
    Morgan fingerprint calculation...
    Done
  Tanimoto similarity calculation...
  Done
  Graph entropy similarity calculation...
  Done
Target: ESR1_ago Process...
  #actives (ESR1_ago): 10
    Morgan fingerprint calculation...
    Done
  #inactives (ESR1_ago): 100
    Morgan fingerprint calculation...
    Done
  Tanimoto similarity calculation...
  Done
  Graph entropy similarity calculation...
  Done
Target: ESR1_ant Process...
  #actives (ESR1_ant): 10
    Morgan fingerprint calculation...
    Done
  #inactives (ESR1_ant): 100
    Morgan fingerprint calculation...
    Done
  Tanimoto similarity calculati

In [45]:
import polars as pl
import random
import glob
import os
from rdkit.ML.Scoring import Scoring

num_columns = len(glob.glob(os.path.join("./result/LIT-PCBA/ge_sim/result_ADRB2/", f"gesims_r1_active*.txt")))
df = pl.read_csv(
    "./result/LIT-PCBA/ge_sim/result_ADRB2/merged_results_r1.txt",
    has_header=False,
    separator="\t",
    new_columns=['index'] + [str(i) for i in range(num_columns)],
    dtypes=[pl.Int32] + [pl.Float32 for _ in range(num_columns)])

column_names = df.columns
column_names.remove('index')
df = df.with_columns((pl.col('index').is_in([int(i) for i in column_names])).alias('is_active'))
sampled_columns = ['0', '2', '8', '4', '11']
idxs = df.select(
        pl.col('index'),
        pl.max_horizontal(sampled_columns)
    ).sort(
        'max',
        descending=True,
    ).filter(
        ~pl.col('index').is_in([int(i) for i in sampled_columns])
    ).select(
        pl.col('index')
    ).limit(
        int(len(df) * 0.05)
    ).to_series()
#bool_list = (df
#    .select(
#        pl.col('index'), 
#        pl.max_horizontal(sampled_columns),
#        pl.col('is_active'))
#    .sort(
#        'max', 
#        descending=True,)
#    .filter(
#        ~pl.col('index').is_in([int(i) for i in sampled_columns]))  # remove training compounds
#    .select(
#        pl.col('is_active'))
#    .to_numpy().tolist()
#)

(5,)

In [59]:
import polars as pl
import random
import glob
import os
from rdkit.ML.Scoring import Scoring

num_columns = len(glob.glob(os.path.join("./result/ge_sim/result_ADRB2/", f"gesims_r1_active*.txt")))
df = pl.read_csv(
    "./result/ge_sim/result_ADRB2/merged_results_r1.txt",
    has_header=False,
    separator='\t',
    new_columns=['index'] + [str(i) for i in range(num_columns)],
    dtypes=[pl.Int32] + [pl.Float32 for _ in range(num_columns)])
column_names = df.columns
column_names.remove('index')
df = df.with_columns((pl.col('index').is_in([int(i) for i in column_names])).alias('is_active'))
active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
sampled_columns = random.sample(active_column_names, 10)
bool_list = (df
    .select(
        pl.col('index'), 
        #pl.max(sampled_columns),
        pl.max_horizontal(sampled_columns),
        pl.col('is_active'))
    .sort(
        'max', 
        descending=True,)
    .filter(
        ~pl.col('index').is_in([int(i) for i in sampled_columns]))  # remove training compounds
    .select(
        pl.col('is_active'))
    .to_numpy().tolist())

print(Scoring.CalcEnrichment(bool_list, 0, fractions=[0.05])[0])
print(Scoring.CalcBEDROC(bool_list, 0, alpha=20))
print(Scoring.CalcAUC(bool_list, 0))
#if useGraphSim:
#    num_columns = len(glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt")))
#    fname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
#else:  # tanimoto
#    num_columns = len(glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt")))
#    fname = os.path.join(dirname, "merged_results.txt")
#df = pl.read_csv(
#    fname,
#    has_header=False,
#    separator="\t",
#    new_columns=['index'] + [str(i) for i in range(num_columns)],
#    dtypes=[pl.Int32] + [pl.Float32 for _ in range(num_columns)])
#
#column_names = df.columns
#column_names.remove('index')
#df = df.with_columns((pl.col('index').is_in([int(i) for i in column_names])).alias('is_active'))
#
#return df

2.5476190476190474
0.19643948307118506
0.5985714285714286


In [46]:
import glob
import os
import pickle
import random
from typing import Dict, List

import polars as pl
import numpy as np
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.ML.Scoring import Scoring


def _calculate_ef_bedroc_auc(
    dirname: str,
    trial_num: int =50,
    sample_num: int =10,
    fractions: List[float] = [0.05],
    alpha: int =20,
    useGraphSim: bool =False,
    radiusGraphSim: int =2) -> Dict[str, List[float]]:

    ef_list = []
    bedroc_list = []
    auc_list = []

    df = _create_dataframe_from_files(
        dirname,
        useGraphSim=useGraphSim,
        radiusGraphSim=radiusGraphSim)
    active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
    for _ in range(trial_num):
        sampled_columns = random.sample(active_column_names, sample_num)
        bool_list = (df
            .select(
                pl.col('index'), 
                pl.max_horizontal(sampled_columns),
                pl.col('is_active'))
            .sort(
                'max', 
                descending=True,)
            .filter(
                ~pl.col('index').is_in([int(i) for i in sampled_columns]))  # remove training compounds
            .select(
                pl.col('is_active'))
            .to_numpy().tolist()
        )
        
        ef_list.append(Scoring.CalcEnrichment(bool_list, 0, fractions=fractions)[0])
        bedroc_list.append(Scoring.CalcBEDROC(bool_list, 0, alpha=alpha))
        auc_list.append(Scoring.CalcAUC(bool_list, 0))
        
    ret_dict = {
        'enrichment_factor': ef_list,
        'BEDROC': bedroc_list,
        'AUC': auc_list,
    }
    return ret_dict

    
def _merge_files(dirname: str, useGraphSim: bool =False, radiusGraphSim: int =2):
    def _extract_number(fname):
        base, _ = os.path.splitext(fname)
        return int(base.split("_active")[-1])

    if useGraphSim:
        files = glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt"))
    else:  # tanimoto
        files = glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt"))
    files_sorted = sorted(files, key=_extract_number)
    print(files_sorted)

    with open(files_sorted[0], 'r') as f:
        num_lines = len(f.readlines())
    merged_results = [[str(i)] for i in range(num_lines)]  # initialization with index

    print("[INFO] Collect results...")
    for fname in files_sorted:
        with open(fname, 'r') as f:
            lines = f.readlines()
        for i, l in enumerate(lines):
            _, value = l.split('\t')
            merged_results[i].append(value.strip('\n'))
    if useGraphSim:
        ofname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
    else:
        ofname = os.path.join(dirname, f"merged_results.txt")
    print(f"[INFO] Dump file: {ofname}")
    with open(ofname, 'w') as f:
        f.write('\n'.join('\t'.join(r) for r in merged_results))


def _create_dataframe_from_files(dirname: str, useGraphSim: bool =False, radiusGraphSim: int =2) -> pl.DataFrame:
    if useGraphSim:
        num_columns = len(glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt")))
        fname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
    else:  # tanimoto
        num_columns = len(glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt")))
        fname = os.path.join(dirname, "merged_results.txt")
    df = pl.read_csv(
        fname,
        has_header=False,
        separator="\t",
        new_columns=['index'] + [str(i) for i in range(num_columns)],
        dtypes=[pl.Int32] + [pl.Float32 for _ in range(num_columns)])

    column_names = df.columns
    column_names.remove('index')
    df = df.with_columns((pl.col('index').is_in([int(i) for i in column_names])).alias('is_active'))
    
    return df


def get_scaffolds_from_target_name(dataset_dir: str) -> np.array(str):
    print(f"Target: {dataset_dir} Process...")
    with open(os.path.join(dataset_dir, "actives.smi"), 'r') as f:
        lines = [l.split(' ')[0] for l in f.readlines()]
        actives = [MurckoScaffold.MurckoScaffoldSmiles(s) for s in lines]
    with open(os.path.join(dataset_dir, "inactives.smi"), 'r') as f:
        lines = [l.split(' ')[0] for l in f.readlines()]
        inactives = [MurckoScaffold.MurckoScaffoldSmiles(s) for s in lines]
    return np.array(actives + inactives)


def _calc_scaffoldEF(
    dirname: str,
    scaffolds: np.array(str),
    fraction: float =0.05,
    sample_num: int =10,
    trial_num: int =50,
    useGraphSim: bool=False,
    radiusGraphSim: int=2)->float:
    
    scaffold_ef_list = []

    df = _create_dataframe_from_files(
        dirname,
        useGraphSim=useGraphSim,
        radiusGraphSim=radiusGraphSim)

    total_num = scaffolds.shape[0]
    total_scaffolds_num = np.unique(scaffolds).shape[0]
    active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
    for _ in range(trial_num):
        sampled_columns = random.sample(active_column_names, sample_num)
        top_n_percent_idxs = df.select(
            pl.col('index'),
            pl.max_horizontal(sampled_columns),
         ).sort(
            'max', descending=True
         ).filter(
            ~pl.col('index').is_in([int(i) for i in sampled_columns])  # remove training compounds
         ).select(
             pl.col('index')
         ).limit(
             int(len(df) * fraction)
         ).to_series()
        subset_num = top_n_percent_idxs.shape[0]
        subset_scaffolds_num = np.unique(scaffolds[top_n_percent_idxs]).shape[0]
        scaffold_ef_list.append((subset_scaffolds_num / subset_num) / (total_scaffolds_num / total_num))
    
    ret_dict = {
        "scaffold_enrichment_factor": scaffold_ef_list
    }
    return ret_dict


def main():
    trial_num = 50
    sample_num = 10
    radius_list = [1, 2, 3, 4]
    dataset_root_dir = "./data/benchmarking_platform/compounds/MUV/"
    result_dir_gesim = "./result/benchmarking_platform/MUV/ge_sim"
    result_dir_tanimoto = "./result/benchmarking_platform/MUV/tanimoto_sim/"
    base_dir_list = [result_dir_gesim, result_dir_tanimoto]
    alpha_list = [20, 100]
    fractions_list = [[0.05], [0.01]]
    do_merge_files = True

    for base_dir in base_dir_list:
        print(base_dir)
        target_dirs = [d for d in glob.glob(os.path.join(base_dir, "result_*")) if os.path.isdir(d)]
        target_dirs.sort()
        
        if base_dir == result_dir_gesim:
            for alpha, fractions in zip(alpha_list, fractions_list):
                for r in radius_list:
                    result_dict = {}
                    for target_dir in target_dirs:
                        target_name = os.path.basename(target_dir).split("_", 1)[1]
                        print(target_name)
                        if do_merge_files:
                            _merge_files(target_dir, useGraphSim=True, radiusGraphSim=r)
                        result = _calculate_ef_bedroc_auc(
                            target_dir,
                            trial_num=trial_num,
                            sample_num=sample_num,
                            alpha=alpha,
                            fractions=fractions,
                            useGraphSim=True,
                            radiusGraphSim=r)
                        scaffolds = get_scaffolds_from_target_name(os.path.join(dataset_root_dir, target_name))
                        result.update(_calc_scaffoldEF(
                            target_dir,
                            scaffolds=scaffolds,
                            fraction=fractions[0],
                            trial_num=trial_num,
                            sample_num=sample_num,
                            useGraphSim=True,
                            radiusGraphSim=r
                        ))
                        result_dict[target_name] = result
                    ofname = f"{base_dir}/result_r{r}_a{alpha}_f{str(fractions[0]).replace('.', '')}.pkl"
                    with open(ofname, 'wb') as f:
                        pickle.dump(result_dict, f)
        else:  # "./result/tanimoto_sim"
            for alpha, fractions in zip(alpha_list, fractions_list):
                result_dict = {}
                for target_dir in target_dirs:
                    target_name = os.path.basename(target_dir).split("_", 1)[1]
                    print(target_name)
                    if do_merge_files:
                        _merge_files(target_dir, useGraphSim=False, radiusGraphSim=None)
                    result = _calculate_ef_bedroc_auc(
                        target_dir,
                        trial_num=trial_num,
                        sample_num=sample_num,
                        alpha=alpha,
                        fractions=fractions,
                        useGraphSim=False,
                        radiusGraphSim=None)
                    scaffolds = get_scaffolds_from_target_name(os.path.join(dataset_root_dir, target_name))
                    result.update(_calc_scaffoldEF(
                        target_dir,
                        scaffolds=scaffolds,
                        fraction=fractions[0],
                        trial_num=trial_num,
                        sample_num=sample_num,
                        useGraphSim=False,
                        radiusGraphSim=None
                    ))
                    result_dict[target_name] = result
                ofname = f"{base_dir}/result_a{alpha}_f{str(fractions[0]).replace('.', '')}.pkl"
                with open(ofname, 'wb') as f:
                    pickle.dump(result_dict, f)

main()

./result/benchmarking_platform/MUV/ge_sim
MUV_466
['./result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active0.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active1.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active2.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active3.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active4.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active5.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active6.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active7.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active8.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active9.txt', './result/benchmarking_platform/MUV/ge_sim/result_MUV_466/gesims_r1_active10.txt', './result/benchmarking_platform/MUV/ge_sim/res

In [ ]:
import glob
import os
import pickle
import random
from typing import Dict, List

import polars as pl
import numpy as np
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.ML.Scoring import Scoring


def _calculate_ef_bedroc_auc(
    dirname: str,
    trial_num: int =50,
    sample_num: int =10,
    fractions: List[float] = [0.05],
    alpha: int =20,
    useGraphSim: bool =False,
    radiusGraphSim: int =2) -> Dict[str, List[float]]:

    ef_list = []
    bedroc_list = []
    auc_list = []

    df = _create_dataframe_from_files(
        dirname,
        useGraphSim=useGraphSim,
        radiusGraphSim=radiusGraphSim)
    active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
    for _ in range(trial_num):
        sampled_columns = random.sample(active_column_names, sample_num)
        bool_list = (df
            .select(
                pl.col('index'), 
                pl.max_horizontal(sampled_columns),
                pl.col('is_active'))
            .sort(
                'max', 
                descending=True,)
            .filter(
                ~pl.col('index').is_in([int(i) for i in sampled_columns]))  # remove training compounds
            .select(
                pl.col('is_active'))
            .to_numpy().tolist()
        )
        
        ef_list.append(Scoring.CalcEnrichment(bool_list, 0, fractions=fractions)[0])
        bedroc_list.append(Scoring.CalcBEDROC(bool_list, 0, alpha=alpha))
        auc_list.append(Scoring.CalcAUC(bool_list, 0))
        
    ret_dict = {
        'enrichment_factor': ef_list,
        'BEDROC': bedroc_list,
        'AUC': auc_list,
    }
    return ret_dict

    
def _merge_files(dirname: str, useGraphSim: bool =False, radiusGraphSim: int =2):
    def _extract_number(fname):
        base, _ = os.path.splitext(fname)
        return int(base.split("_active")[-1])

    if useGraphSim:
        files = glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt"))
    else:  # tanimoto
        files = glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt"))
    files_sorted = sorted(files, key=_extract_number)
    print(files_sorted)

    with open(files_sorted[0], 'r') as f:
        num_lines = len(f.readlines())
    merged_results = [[str(i)] for i in range(num_lines)]  # initialization with index

    print("[INFO] Collect results...")
    for fname in files_sorted:
        with open(fname, 'r') as f:
            lines = f.readlines()
        for i, l in enumerate(lines):
            _, value = l.split('\t')
            merged_results[i].append(value.strip('\n'))
    if useGraphSim:
        ofname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
    else:
        ofname = os.path.join(dirname, f"merged_results.txt")
    print(f"[INFO] Dump file: {ofname}")
    with open(ofname, 'w') as f:
        f.write('\n'.join('\t'.join(r) for r in merged_results))


def _create_dataframe_from_files(dirname: str, useGraphSim: bool =False, radiusGraphSim: int =2) -> pl.DataFrame:
    if useGraphSim:
        num_columns = len(glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt")))
        fname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
    else:  # tanimoto
        num_columns = len(glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt")))
        fname = os.path.join(dirname, "merged_results.txt")
    df = pl.read_csv(
        fname,
        has_header=False,
        separator="\t",
        new_columns=['index'] + [str(i) for i in range(num_columns)],
        dtypes=[pl.Int32] + [pl.Float32 for _ in range(num_columns)])

    column_names = df.columns
    column_names.remove('index')
    df = df.with_columns((pl.col('index').is_in([int(i) for i in column_names])).alias('is_active'))
    
    return df


def get_scaffolds_from_target_name(dataset_dir: str) -> np.array(str):
    print(f"Target: {dataset_dir} Process...")
    with open(os.path.join(dataset_dir, "actives.smi"), 'r') as f:
        lines = [l.split(' ')[0] for l in f.readlines()]
        actives = [MurckoScaffold.MurckoScaffoldSmiles(s) for s in lines]
    with open(os.path.join(dataset_dir, "inactives.smi"), 'r') as f:
        lines = [l.split(' ')[0] for l in f.readlines()]
        inactives = [MurckoScaffold.MurckoScaffoldSmiles(s) for s in lines]
    return np.array(actives + inactives)


def _calc_scaffoldEF(
    dirname: str,
    scaffolds: np.array(str),
    fraction: float =0.05,
    sample_num: int =10,
    trial_num: int =50,
    useGraphSim: bool=False,
    radiusGraphSim: int=2)->float:
    
    scaffold_ef_list = []

    df = _create_dataframe_from_files(
        dirname,
        useGraphSim=useGraphSim,
        radiusGraphSim=radiusGraphSim)

    total_num = scaffolds.shape[0]
    total_scaffolds_num = np.unique(scaffolds).shape[0]
    active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
    for _ in range(trial_num):
        sampled_columns = random.sample(active_column_names, sample_num)
        top_n_percent_idxs = df.select(
            pl.col('index'),
            pl.max_horizontal(sampled_columns),
         ).sort(
            'max', descending=True
         ).filter(
            ~pl.col('index').is_in([int(i) for i in sampled_columns])  # remove training compounds
         ).select(
             pl.col('index')
         ).limit(
             int(len(df) * fraction)
         ).to_series()
        subset_num = top_n_percent_idxs.shape[0]
        subset_scaffolds_num = np.unique(scaffolds[top_n_percent_idxs]).shape[0]
        scaffold_ef_list.append((subset_scaffolds_num / subset_num) / (total_scaffolds_num / total_num))
    
    ret_dict = {
        "scaffold_enrichment_factor": scaffold_ef_list
    }
    return ret_dict


def main():
    trial_num = 50
    sample_num = 10
    radius_list = [1, 2, 3, 4]
    #dataset_root_dir = "./data/benchmarking_platform/compounds/MUV/"
    #result_dir_gesim = "./result/benchmarking_platform/MUV/ge_sim"
    #result_dir_tanimoto = "./result/benchmarking_platform/MUV/tanimoto_sim/"
    dataset_root_dir = "./data/benchmarking_platform/compounds/ChEMBL"
    result_dir_gesim = "./result/benchmarking_platform/ChEMBL /ge_sim"
    result_dir_tanimoto = "./result/benchmarking_platform/ChEMBL /tanimoto_sim/"
    base_dir_list = [result_dir_gesim, result_dir_tanimoto]
    alpha_list = [20, 100]
    fractions_list = [[0.05], [0.01]]
    do_merge_files = True

    for base_dir in base_dir_list:
        print(base_dir)
        target_dirs = [d for d in glob.glob(os.path.join(base_dir, "result_*")) if os.path.isdir(d)]
        target_dirs.sort()
        
        if base_dir == result_dir_gesim:
            for alpha, fractions in zip(alpha_list, fractions_list):
                for r in radius_list:
                    result_dict = {}
                    for target_dir in target_dirs:
                        target_name = os.path.basename(target_dir).split("_", 1)[1]
                        print(target_name)
                        if do_merge_files:
                            _merge_files(target_dir, useGraphSim=True, radiusGraphSim=r)
                        result = _calculate_ef_bedroc_auc(
                            target_dir,
                            trial_num=trial_num,
                            sample_num=sample_num,
                            alpha=alpha,
                            fractions=fractions,
                            useGraphSim=True,
                            radiusGraphSim=r)
                        scaffolds = get_scaffolds_from_target_name(os.path.join(dataset_root_dir, target_name))
                        result.update(_calc_scaffoldEF(
                            target_dir,
                            scaffolds=scaffolds,
                            fraction=fractions[0],
                            trial_num=trial_num,
                            sample_num=sample_num,
                            useGraphSim=True,
                            radiusGraphSim=r
                        ))
                        result_dict[target_name] = result
                    ofname = f"{base_dir}/result_r{r}_a{alpha}_f{str(fractions[0]).replace('.', '')}.pkl"
                    with open(ofname, 'wb') as f:
                        pickle.dump(result_dict, f)
        else:  # "./result/tanimoto_sim"
            for alpha, fractions in zip(alpha_list, fractions_list):
                result_dict = {}
                for target_dir in target_dirs:
                    target_name = os.path.basename(target_dir).split("_", 1)[1]
                    print(target_name)
                    if do_merge_files:
                        _merge_files(target_dir, useGraphSim=False, radiusGraphSim=None)
                    result = _calculate_ef_bedroc_auc(
                        target_dir,
                        trial_num=trial_num,
                        sample_num=sample_num,
                        alpha=alpha,
                        fractions=fractions,
                        useGraphSim=False,
                        radiusGraphSim=None)
                    scaffolds = get_scaffolds_from_target_name(os.path.join(dataset_root_dir, target_name))
                    result.update(_calc_scaffoldEF(
                        target_dir,
                        scaffolds=scaffolds,
                        fraction=fractions[0],
                        trial_num=trial_num,
                        sample_num=sample_num,
                        useGraphSim=False,
                        radiusGraphSim=None
                    ))
                    result_dict[target_name] = result
                ofname = f"{base_dir}/result_a{alpha}_f{str(fractions[0]).replace('.', '')}.pkl"
                with open(ofname, 'wb') as f:
                    pickle.dump(result_dict, f)

main()

./result/benchmarking_platform/ChEMBL /ge_sim
ChEMBL_100
['./result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active0.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active1.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active2.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active3.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active4.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active5.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active6.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active7.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active8.txt', './result/benchmarking_platform/ChEMBL /ge_sim/result_ChEMBL_100/gesims_r1_active9.txt', './result/benchmarking_platform/ChEMBL /ge_sim/resul

In [ ]:
import glob
import os
import pickle
import random
from typing import Dict, List

import polars as pl
import numpy as np
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.ML.Scoring import Scoring


def _calculate_ef_bedroc_auc(
    dirname: str,
    trial_num: int =50,
    sample_num: int =10,
    fractions: List[float] = [0.05],
    alpha: int =20,
    useGraphSim: bool =False,
    radiusGraphSim: int =2) -> Dict[str, List[float]]:

    ef_list = []
    bedroc_list = []
    auc_list = []

    df = _create_dataframe_from_files(
        dirname,
        useGraphSim=useGraphSim,
        radiusGraphSim=radiusGraphSim)
    active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
    for _ in range(trial_num):
        sampled_columns = random.sample(active_column_names, sample_num)
        bool_list = (df
            .select(
                pl.col('index'), 
                pl.max_horizontal(sampled_columns),
                pl.col('is_active'))
            .sort(
                'max', 
                descending=True,)
            .filter(
                ~pl.col('index').is_in([int(i) for i in sampled_columns]))  # remove training compounds
            .select(
                pl.col('is_active'))
            .to_numpy().tolist()
        )
        
        ef_list.append(Scoring.CalcEnrichment(bool_list, 0, fractions=fractions)[0])
        bedroc_list.append(Scoring.CalcBEDROC(bool_list, 0, alpha=alpha))
        auc_list.append(Scoring.CalcAUC(bool_list, 0))
        
    ret_dict = {
        'enrichment_factor': ef_list,
        'BEDROC': bedroc_list,
        'AUC': auc_list,
    }
    return ret_dict

    
def _merge_files(dirname: str, useGraphSim: bool =False, radiusGraphSim: int =2):
    def _extract_number(fname):
        base, _ = os.path.splitext(fname)
        return int(base.split("_active")[-1])

    if useGraphSim:
        files = glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt"))
    else:  # tanimoto
        files = glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt"))
    files_sorted = sorted(files, key=_extract_number)
    print(files_sorted)

    with open(files_sorted[0], 'r') as f:
        num_lines = len(f.readlines())
    merged_results = [[str(i)] for i in range(num_lines)]  # initialization with index

    print("[INFO] Collect results...")
    for fname in files_sorted:
        with open(fname, 'r') as f:
            lines = f.readlines()
        for i, l in enumerate(lines):
            _, value = l.split('\t')
            merged_results[i].append(value.strip('\n'))
    if useGraphSim:
        ofname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
    else:
        ofname = os.path.join(dirname, f"merged_results.txt")
    print(f"[INFO] Dump file: {ofname}")
    with open(ofname, 'w') as f:
        f.write('\n'.join('\t'.join(r) for r in merged_results))


def _create_dataframe_from_files(dirname: str, useGraphSim: bool =False, radiusGraphSim: int =2) -> pl.DataFrame:
    if useGraphSim:
        num_columns = len(glob.glob(os.path.join(dirname, f"gesims_r{radiusGraphSim}_active*.txt")))
        fname = os.path.join(dirname, f"merged_results_r{radiusGraphSim}.txt")
    else:  # tanimoto
        num_columns = len(glob.glob(os.path.join(dirname, f"tanimoto_sims_active*.txt")))
        fname = os.path.join(dirname, "merged_results.txt")
    df = pl.read_csv(
        fname,
        has_header=False,
        separator="\t",
        new_columns=['index'] + [str(i) for i in range(num_columns)],
        dtypes=[pl.Int32] + [pl.Float32 for _ in range(num_columns)])

    column_names = df.columns
    column_names.remove('index')
    df = df.with_columns((pl.col('index').is_in([int(i) for i in column_names])).alias('is_active'))
    
    return df


def get_scaffolds_from_target_name(dataset_dir: str) -> np.array(str):
    print(f"Target: {dataset_dir} Process...")
    with open(os.path.join(dataset_dir, "actives.smi"), 'r') as f:
        lines = [l.split(' ')[0] for l in f.readlines()]
        actives = [MurckoScaffold.MurckoScaffoldSmiles(s) for s in lines]
    with open(os.path.join(dataset_dir, "inactives.smi"), 'r') as f:
        lines = [l.split(' ')[0] for l in f.readlines()]
        inactives = [MurckoScaffold.MurckoScaffoldSmiles(s) for s in lines]
    return np.array(actives + inactives)


def _calc_scaffoldEF(
    dirname: str,
    scaffolds: np.array(str),
    fraction: float =0.05,
    sample_num: int =10,
    trial_num: int =50,
    useGraphSim: bool=False,
    radiusGraphSim: int=2)->float:
    
    scaffold_ef_list = []

    df = _create_dataframe_from_files(
        dirname,
        useGraphSim=useGraphSim,
        radiusGraphSim=radiusGraphSim)

    total_num = scaffolds.shape[0]
    total_scaffolds_num = np.unique(scaffolds).shape[0]
    active_column_names = [c for c in df.columns if c not in {'index', 'is_active'}]
    for _ in range(trial_num):
        sampled_columns = random.sample(active_column_names, sample_num)
        top_n_percent_idxs = df.select(
            pl.col('index'),
            pl.max_horizontal(sampled_columns),
         ).sort(
            'max', descending=True
         ).filter(
            ~pl.col('index').is_in([int(i) for i in sampled_columns])  # remove training compounds
         ).select(
             pl.col('index')
         ).limit(
             int(len(df) * fraction)
         ).to_series()
        subset_num = top_n_percent_idxs.shape[0]
        subset_scaffolds_num = np.unique(scaffolds[top_n_percent_idxs]).shape[0]
        scaffold_ef_list.append((subset_scaffolds_num / subset_num) / (total_scaffolds_num / total_num))
    
    ret_dict = {
        "scaffold_enrichment_factor": scaffold_ef_list
    }
    return ret_dict


def main():
    trial_num = 50
    sample_num = 10
    radius_list = [1, 2, 3, 4]
    #dataset_root_dir = "./data/benchmarking_platform/compounds/MUV/"
    #result_dir_gesim = "./result/benchmarking_platform/MUV/ge_sim"
    #result_dir_tanimoto = "./result/benchmarking_platform/MUV/tanimoto_sim/"
    dataset_root_dir = "./data/benchmarking_platform/compounds/DUD"
    result_dir_gesim = "./result/benchmarking_platform/DUD/ge_sim"
    result_dir_tanimoto = "./result/benchmarking_platform/DUD/tanimoto_sim/"
    base_dir_list = [result_dir_gesim, result_dir_tanimoto]
    alpha_list = [20, 100]
    fractions_list = [[0.05], [0.01]]
    do_merge_files = True

    for base_dir in base_dir_list:
        print(base_dir)
        target_dirs = [d for d in glob.glob(os.path.join(base_dir, "result_*")) if os.path.isdir(d)]
        target_dirs.sort()
        
        if base_dir == result_dir_gesim:
            for alpha, fractions in zip(alpha_list, fractions_list):
                for r in radius_list:
                    result_dict = {}
                    for target_dir in target_dirs:
                        target_name = os.path.basename(target_dir).split("_", 1)[1]
                        print(target_name)
                        if do_merge_files:
                            _merge_files(target_dir, useGraphSim=True, radiusGraphSim=r)
                        result = _calculate_ef_bedroc_auc(
                            target_dir,
                            trial_num=trial_num,
                            sample_num=sample_num,
                            alpha=alpha,
                            fractions=fractions,
                            useGraphSim=True,
                            radiusGraphSim=r)
                        scaffolds = get_scaffolds_from_target_name(os.path.join(dataset_root_dir, target_name))
                        result.update(_calc_scaffoldEF(
                            target_dir,
                            scaffolds=scaffolds,
                            fraction=fractions[0],
                            trial_num=trial_num,
                            sample_num=sample_num,
                            useGraphSim=True,
                            radiusGraphSim=r
                        ))
                        result_dict[target_name] = result
                    ofname = f"{base_dir}/result_r{r}_a{alpha}_f{str(fractions[0]).replace('.', '')}.pkl"
                    with open(ofname, 'wb') as f:
                        pickle.dump(result_dict, f)
        else:  # "./result/tanimoto_sim"
            for alpha, fractions in zip(alpha_list, fractions_list):
                result_dict = {}
                for target_dir in target_dirs:
                    target_name = os.path.basename(target_dir).split("_", 1)[1]
                    print(target_name)
                    if do_merge_files:
                        _merge_files(target_dir, useGraphSim=False, radiusGraphSim=None)
                    result = _calculate_ef_bedroc_auc(
                        target_dir,
                        trial_num=trial_num,
                        sample_num=sample_num,
                        alpha=alpha,
                        fractions=fractions,
                        useGraphSim=False,
                        radiusGraphSim=None)
                    scaffolds = get_scaffolds_from_target_name(os.path.join(dataset_root_dir, target_name))
                    result.update(_calc_scaffoldEF(
                        target_dir,
                        scaffolds=scaffolds,
                        fraction=fractions[0],
                        trial_num=trial_num,
                        sample_num=sample_num,
                        useGraphSim=False,
                        radiusGraphSim=None
                    ))
                    result_dict[target_name] = result
                ofname = f"{base_dir}/result_a{alpha}_f{str(fractions[0]).replace('.', '')}.pkl"
                with open(ofname, 'wb') as f:
                    pickle.dump(result_dict, f)

main()

In [41]:
target_dirs = [d for d in glob.glob(os.path.join('result/LIT-PCBA/ge_sim', "result_*")) if os.path.isdir(d)]
for t in target_dirs:
    print(os.path.basename(t).split("_", 1))

['result', 'ADRB2']
['result', 'ALDH1']
['result', 'ESR1_ago']
['result', 'ESR1_ant']
['result', 'FEN1']
['result', 'GBA']
['result', 'IDH1']
['result', 'KAT2A']
['result', 'MAPK1']
['result', 'MTORC1']
['result', 'OPRK1']
['result', 'PKM2']
['result', 'PPARG']
['result', 'TP53']
['result', 'VDR']
